In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import data_cleaning

In [2]:
billionaires = pd.read_csv('billionaires.csv')
ed_stats_country = pd.read_csv('EdStatsCountry.csv')

In [3]:
import pandas as pd

def merge_billionaires_ed_stats(billionaires, ed_stats_country):
    # Mapping of country names from 'billionaires' to 'ed_stats_country' 'Short Name'
    country_mapping = {
        'Hong Kong': 'Hong Kong SAR, China',
        'South Korea': 'Korea',
        'Eswatini (Swaziland)': 'Swaziland',
        'Bahamas': 'The Bahamas',
        'British Virgin Islands': 'Virgin Islands',
        'Guernsey': 'Channel Islands',
        'Slovakia': 'Slovak Republic',
    }
    
    # Apply the mapping to the 'country' column in the billionaires DataFrame
    billionaires['country'] = billionaires['country'].map(country_mapping).fillna(billionaires['country'])
    
    # Merge the DataFrames
    merged_df = pd.merge(billionaires, ed_stats_country, left_on='country', right_on='Short Name', how='left')
    
    return merged_df

In [5]:
merged_df = merge_billionaires_ed_stats(billionaires, ed_stats_country)
merged_df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,Unnamed: 31
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,Special Data Dissemination Standard (SDDS),2006. Rolling census based on continuous sampl...,NaN,"Expenditure survey/budget survey (ES/BS), 1994/95",Yes,2010,2009.0,2012.0,2007,NaN
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,Special Data Dissemination Standard (SDDS),2010,NaN,"Labor force survey (LFS), 2000",Yes,2012,2008.0,2012.0,2005,NaN
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,Special Data Dissemination Standard (SDDS),2010,NaN,"Labor force survey (LFS), 2000",Yes,2012,2008.0,2012.0,2005,NaN
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,Special Data Dissemination Standard (SDDS),2010,NaN,"Labor force survey (LFS), 2000",Yes,2012,2008.0,2012.0,2005,NaN
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,Special Data Dissemination Standard (SDDS),2010,NaN,"Labor force survey (LFS), 2000",Yes,2012,2008.0,2012.0,2005,NaN


In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2640 entries, 0 to 2639
Data columns (total 67 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   rank                                               2640 non-null   int64  
 1   finalWorth                                         2640 non-null   int64  
 2   category                                           2640 non-null   object 
 3   personName                                         2640 non-null   object 
 4   age                                                2575 non-null   float64
 5   country                                            2602 non-null   object 
 6   city                                               2568 non-null   object 
 7   source                                             2640 non-null   object 
 8   industries                                         2640 non-null   object 
 9   countryO

In [7]:
# drop columns with too many nulls (2000+) or redundant
merged_df = merged_df.drop(columns=['organization', 'title', 'Other groups', 'Vital registration complete', 'Alternative conversion factor', 'Unnamed: 31'])

In [8]:
#convert datetime columns
placeholder_date = pd.Timestamp('2024-03-01')
merged_df['birthDate'] = pd.to_datetime(merged_df['birthDate']).fillna(placeholder_date)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['birthYear'] = merged_df['birthYear'].fillna(-1).astype(int)
merged_df['birthMonth'] = merged_df['birthMonth'].fillna(-1).astype(int)
merged_df['birthDay'] = merged_df['birthDay'].fillna(-1).astype(int)
merged_df['National accounts reference year'] = merged_df['National accounts reference year'].fillna(-1).astype(int)
uncleaned_dates = ['Latest population census', 'Latest agricultural census', 'Latest industrial data', 'Latest trade data',
                  'Latest water withdrawal data']
for col in uncleaned_dates:
    merged_df[col] = merged_df[col].astype(str).str.extract('(\d{4})')
    merged_df[col] = pd.to_datetime(merged_df[col], format='%Y', errors='coerce').fillna(placeholder_date)

#convert numerical columns
numerical_columns = ['age', 'cpi_country', 'cpi_change_country', 'gdp_country', 'gross_tertiary_education_enrollment', 
                     'gross_primary_education_enrollment_country', 'life_expectancy_country', 'tax_revenue_country_country',
                     'total_tax_rate_country', 'population_country', 'latitude_country', 'longitude_country', 'Currency Unit']
for col in numerical_columns:
    merged_df[col] = merged_df[col].fillna(merged_df['age'].mean())

#categorical columns
categorical_columns = ['category', 'city', 'country', 'state', 'gender', 'status', 'residenceStateRegion', 'source', 'industries', 
                       'countryOfCitizenship', 'Country Code', 'Short Name', 'Table Name', 'Long Name', '2-alpha code', 'Region', 
                      'Income Group', 'WB-2 code', 'National accounts base year', 'SNA price valuation', 'Lending category',
                      'System of National Accounts', 'PPP survey year', 'External debt Reporting status', 'System of trade', 
                      'Government Accounting concept', 'IMF data dissemination standard', 'Latest household survey', 
                      'Source of most recent Income and expenditure data', 'Balance of Payments Manual in use']
for col in categorical_columns:
    merged_df[col] = merged_df[col].astype('category').cat.add_categories(['Unknown'])
    merged_df[col] = merged_df[col].fillna('Unknown')

#fill nan for remaining columns
merged_df['firstName'] = merged_df['firstName'].fillna('Unknown')
merged_df['Special Notes'] = merged_df['Special Notes'].fillna('None')


In [9]:
missing_values = merged_df.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0]
columns_with_missing_values

Series([], dtype: int64)